In [162]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame for CMOA Artist Data

In [163]:
csv_file = "Resources/cmoa_artist.csv"
cmoa_artist_df = pd.read_csv(csv_file)
cmoa_artist_df.tail(100)

,artist_id,artist_name,artist_role,nationality,gender,birth_year,birth_place,death_year,death_place,wiki,ulan
28169,NaN,Morison|Tupperware U.S. Inc. Cousins,designer|manufacturer,American|American,NaN,1934.0,NaN,2001.0,|,NaN,NaN
28170,NaN,Morison|Tupperware U.S. Inc. Cousins,designer|manufacturer,American|American,NaN,1934.0,NaN,2001.0,|,NaN,NaN
28171,NaN,Morison|Tupperware U.S. Inc. Cousins,designer|manufacturer,American|American,NaN,1934.0,NaN,2001.0,|,NaN,NaN
28172,NaN,Morison|Tupperware U.S. Inc. Cousins,designer|manufacturer,American|American,NaN,1934.0,NaN,2001.0,|,NaN,NaN
28173,NaN,Morison|Tupperware U.S. Inc. Cousins,designer|manufacturer,American|American,NaN,1934.0,NaN,2001.0,|,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
28264,NaN,Jules Bastien-Lepage,NaN,French,NaN,1848.0,NaN,1884.0,"France, Paris",NaN,NaN
28265,NaN,Pascal A. Dagnan-Bouveret,NaN,French,NaN,1852.0,NaN,1929.0,NaN,NaN,NaN
28266,NaN,George P. Healy,NaN,American,NaN,1813.0,NaN,1894.0,"Chicago (Cook county, Illinois, United States)",NaN,NaN
28267,NaN,Alfred Sisley,NaN,French,NaN,1839.0,NaN,1899.0,"Moret-sur-Loing (Île-de-France, France)",NaN,NaN


In [164]:
# list(cmoa_artist_df)

### Store CSV into DataFrame for CMOA Artwork Data

In [165]:
csv_file = "Resources/cmoa_artwork.csv"
cmoa_artwork_df = pd.read_csv(csv_file)
cmoa_artwork_df.tail(2)

,acc_num,title,medium,acq,create_year,acq_date,copyright,thumb_url,web_url,artist_name,artwork_id,artist_id,museum_code,ulan,wiki,dept,donor_name,classification,donor_type
28267,99.7,View of Saint-Mammès,oil on canvas,NaN,c. 1881,1899-01-09,NaN,http://collection.cmoa.org/CollectionImage.asp...,http://collection.cmoa.org/CollectionDetail.as...,NaN,NaN,NaN,1,NaN,NaN,Fine Arts,"Alfred Sisley [1839-1899], until July 23, 1881...",paintings,Person
28268,99.9,The Clouded Sun,oil on canvas,NaN,1891,1899-02-13,NaN,http://collection.cmoa.org/CollectionImage.asp...,http://collection.cmoa.org/CollectionDetail.as...,NaN,NaN,NaN,1,NaN,NaN,Fine Arts,"Thomas B. Clarke, New York, NY, until 1899; so...",paintings,Person


## CMOA was a datafile where birthdates and deathdates might be:
M/D/Y
Y-M-D
Y-M-D|Y-M-D (multiple artists)

Corrected in Excel:

1) Text to columns (Delimited |) to remove 2nd, 3rd dates.(export to next col)

2) Text to columns (Delimited /) to separate year column for m/y/d

3) Text to columns (delimited -) to separate year column for y-m-d

4) New column collects years: (=IF(E3="",F3,E3))

5) Remove the 0 from blank rows: (=IF(G3=0,"",G3))

Result was a single birth or death year to match other data sources 

### Make edits to cmoa artwork to match artwork dataframes before merge

In [173]:
# # # Drop the 'copyright' and 'wiki' columns
# cmoa_artwork_clean_df = cmoa_artwork_df.drop(['copyright', 'wiki'], axis='columns', inplace=True)

# # # Examine the shape of the DataFrame
# cmoa_artwork_clean_df.head()

In [169]:
cmoa_artwork_df.head(2)

,acc_num,title,medium,acq,create_year,acq_date,thumb_url,web_url,artist_name,artwork_id,artist_id,museum_code,ulan,dept,donor_name,classification,donor_type
0,2002.17,Keith Haring,gelatin silver print,NaN,1984,6/6/2002,http://collection.cmoa.org/CollectionImage.asp...,http://collection.cmoa.org/CollectionDetail.as...,Robert Mapplethorpe,NaN,NaN,1,NaN,Contemporary Art,"The Robert Mapplethorpe Foundation, 2002",photographs,Person
1,2002.19,Untitled,oil on canvas,NaN,1964-1965,6/6/2002,http://collection.cmoa.org/CollectionImage.asp...,http://collection.cmoa.org/CollectionDetail.as...,Jo Baer,NaN,NaN,1,NaN,Contemporary Art,"(Daniel Weinberg Gallery), Santa Monica, CA; M...",paintings,Person


In [170]:
# # Using DataFrame.insert() to add donor_id column 
# do no usedonor_id={}

# do not use cmoa_artwork_df['donor_id']= donor_id 
cmoa_artwork_clean_df = cmoa_artwork_df.assign(donor_id="")

In [171]:
cmoa_artwork_clean_df.tail(2)

,acc_num,title,medium,acq,create_year,acq_date,thumb_url,web_url,artist_name,artwork_id,artist_id,museum_code,ulan,dept,donor_name,classification,donor_type,donor_id
28267,99.7,View of Saint-Mammès,oil on canvas,NaN,c. 1881,1899-01-09,http://collection.cmoa.org/CollectionImage.asp...,http://collection.cmoa.org/CollectionDetail.as...,NaN,NaN,NaN,1,NaN,Fine Arts,"Alfred Sisley [1839-1899], until July 23, 1881...",paintings,Person,
28268,99.9,The Clouded Sun,oil on canvas,NaN,1891,1899-02-13,http://collection.cmoa.org/CollectionImage.asp...,http://collection.cmoa.org/CollectionDetail.as...,NaN,NaN,NaN,1,NaN,Fine Arts,"Thomas B. Clarke, New York, NY, until 1899; so...",paintings,Person,


In [172]:
list(cmoa_artwork_clean_df)

['acc_num',
 'title',
 'medium',
 'acq',
 'create_year',
 'acq_date',
 'thumb_url',
 'web_url',
 'artist_name',
 'artwork_id',
 'artist_id',
 'museum_code',
 'ulan',
 'dept',
 'donor_name',
 'classification',
 'donor_type',
 'donor_id']

# *** Location where move from CMOA work to Artwork work